# E-commerce Sales Analysis Pipeline
**Objective:** Clean and analyze a large-scale retail dataset to extract business KPIs.<br>
**Author:** [Rafael Campos Andrés]<br>
**Date:** January 2026

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [20]:
# Loading raw data
base_dir = Path.cwd().parent
data_path = base_dir / "data" / "raw_sales_data.csv"
if data_path.exists():
    try:
        df = pd.read_csv(data_path)
        print(f"File: {data_path} uploaded succesfully")
    except:
        print(f"File: {data_path} is empty or could not be found")
else:
    print(f"File: {data_path} could not be found")

File: c:\Users\USUARIO\Desktop\GITHUB\E-commerce-Data-Pipeline-Analysis\data\raw_sales_data.csv is empty or could not be found
